# Capstone Project Week 3

In this notebook all the threee parts of the assignment are included.

#### Include and setup of globals

In [46]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

# library to process xml
import xml

# library to handle JSON files
import json

# library to handle requests
import requests 

# import k-means from clustering stage
from sklearn.cluster import KMeans 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# !conda install -c conda-forge geocoder --yes
import geocoder

# !conda install -c conda-forge geopy --yes
import geopy
from geopy.geocoders import Nominatim 

print('Libraries imported.')


# your Foursquare ID
CLIENT_ID = '-'
# your Foursquare Secret
CLIENT_SECRET = '-' 
# Foursquare API version
VERSION = '20180605' 

FOURSCARE_FULL_ONLINE = False

TORONTO_LATITUDE  = 43.6529
TORONTO_LONGITUDE = -79.3849

print('Globals setup.')

Libraries imported.
Globals setup.


# Capstone Project Week 3 - Part 1

## Assignment

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

3. To create the above dataframe:

    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

## Get Data about Toronto

#### Initialize Web Scraper and pull data from Wikipedia page

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

#### Extract html tags for table \< table\> and rows \< tr\>

In [3]:
#Find table
table = soup.find('table')

#Find ands extract rows
rows = []
table_rows = table.select('tr')
for row in table_rows:
    rows.append(row.get_text())

print("Rows loaded: ",len(rows))

Rows loaded:  288


#### Build a pandas dataframe from loaded rows, split entries into columns and name columns

_**Note:** dataframe has two extra columns_

In [4]:
#Build dataframe, split columns and update columns
df_str = pd.DataFrame(rows)
df_raw = df_str[0].str.split('\n', expand=True)

#Assign names to columns and remove header row
df_raw.rename(columns=df_raw.iloc[0], inplace=True)
df_raw.drop(df_raw.index[0], inplace=True)

print('Shape:', df_raw.shape)
df_raw.head(8)

Shape: (287, 5)


,,Postcode,Borough,Neighbourhood,
1,,M1A,Not assigned,Not assigned,
2,,M2A,Not assigned,Not assigned,
3,,M3A,North York,Parkwoods,
4,,M4A,North York,Victoria Village,
5,,M5A,Downtown Toronto,Harbourfront,
6,,M6A,North York,Lawrence Heights,
7,,M6A,North York,Lawrence Manor,
8,,M7A,Queen's Park,Not assigned,


#### Extract columns required

_**Note:** two extracted columns removed_

In [5]:
#Extract relevant columns
df_pure = df_raw[['Postcode', 'Borough', 'Neighbourhood']]

#Adjust column naming
df_pure.columns = ['Postcode', 'Borough', 'Neighborhood']

print('Shape:', df_pure.shape)
df_pure.head(8)

Shape: (287, 3)


,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned


#### Drop rows which have "Not Assigned" in Borough

In [6]:
# Filter entries which have 'Not assigned' in Borough column
borough_notassigned = df_pure[df_pure['Borough'] == 'Not assigned'].index
    
# Delete these row indexes from dataFrame
df_clean = df_pure.drop(borough_notassigned)

print('Shape:', df_clean.shape)
df_clean.head(8)

Shape: (210, 3)


,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Downtown Toronto,Queen's Park
11,M1B,Scarborough,Rouge


#### Set Neighborhoods with value "Not Assigned" to the value of the Borough

In [7]:
#Replace Not assigned in Neighborhood to value of Borough
df_clean.loc[(df_clean.Neighborhood == 'Not assigned'),'Neighborhood'] = df_clean.Borough

print('Shape:', df_clean.shape)
df_clean.head(8)

Shape: (210, 3)


,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Downtown Toronto,Queen's Park
11,M1B,Scarborough,Rouge


#### Group data on Postalcode and Borough

In [8]:
#Merge same PostCodes
df_grouped = df_clean.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

print('Shape:', df_grouped.shape)
df_grouped.head(8)

Shape: (103, 3)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


#### Save dataframe in CSV 

In [9]:
df_grouped.to_csv('capstone-data-package-part-1.csv', index = False)
print("Saved.")

Saved.


# Capstone Project  Week 3 - Part 2

## Assignment

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Given that the geocoder package has been experienced as unreliable, data were comlpemented by the csv file here: http://cocl.us/Geospatial_data to create a dataframe as follows:

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

## Prepare Exploration of Toronto

#### Load cleansed dataset from csv file

In [10]:
df_cleansed = pd.read_csv('capstone-data-package-part-1.csv')
df_cleansed.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Load geospatial data

_**Note:** Alternative approach instead of first download file is direct download (code commented out)_

In [11]:
!wget -q -O 'geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded...')

df_geo = pd.read_csv('geospatial_data.csv')
df_geo.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df_geo.head()

Data downloaded...


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge geospatial data with Boroughs along the Postcode

In [12]:
df_extended = pd.merge(df_cleansed, df_geo, on='Postcode')
print('Data extended...')
print('Shape:', df_extended.shape)
df_extended.head(8)

Data extended...
Shape: (103, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577


#### Save dataframe into CSV

In [13]:
df_extended.to_csv('capstone-data-package-part-2.csv', index = False)
print("Saved.")

Saved.


# Capstone Project Week 3 - Part 3

## Assignment

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together. 

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

## Explore Toronto's Bouroughs and Neighborhoods

#### Load cleansed dataset from csv file

In [14]:
neighborhoods = pd.read_csv('capstone-data-package-part-2.csv')
neighborhoods.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Preprare helper function to build a map using folium

In [15]:
def draw_map_and_neighborhood(df,latitude, longitude):
    
        map = folium.Map(location=[latitude, longitude], zoom_start=11)

        # draw markers on map
        for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
            
            label = '{}, {}'.format(neighborhood, borough)
            
            label = folium.Popup(label, parse_html=True)
            
            folium.CircleMarker(
                [lat, lng],
                radius=5,
                popup=label,
                color='green',
                fill=True,
                fill_color='#31cc77',
                fill_opacity=0.7,
                parse_html=False).add_to(map)  
            
        return map

#### Build Toronto Map with Neighborhoods

In [16]:
# evaluate a center of venues
latitude = neighborhoods['Latitude'].median()
longitude = neighborhoods['Longitude'].median()

draw_map_and_neighborhood(neighborhoods, latitude, longitude)

#### Select Neigborhood in Boroughs with keyword 'Toronto'

In [17]:
selected_neighborhood = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]
selected_neighborhood.reset_index(drop=True, inplace=True)
print('Shape:',selected_neighborhood.shape)
selected_neighborhood.head()

Shape: (39, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Display selected neighborhood on the map

In [18]:
# evaluate a center of venues
latitude = selected_neighborhood['Latitude'].median()
longitude = selected_neighborhood['Longitude'].median()

draw_map_and_neighborhood(selected_neighborhood, latitude, longitude)

## Initial exploration

#### Select first neighberhood

In [19]:
#get center of bourgouh
select_id = 0
selected_neighborhood.loc[select_id]

Postcode                 M4E
Borough         East Toronto
Neighborhood     The Beaches
Latitude             43.6764
Longitude            -79.293
Name: 0, dtype: object

#### Pull data from foursquare

In [48]:
LIMIT = 50
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET,
        VERSION,
        selected_neighborhood.loc[select_id, 'Latitude'],
        selected_neighborhood.loc[select_id, 'Longitude'],
        radius,
        LIMIT)

print(url)

#Removed for publishing
#results = requests.get(url).json()
#results

https://api.foursquare.com/v2/venues/explore?&client_id=-&client_secret=-&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=50


#### Build function to clean up specific result rows

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Process result received from foursquare

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.columns = ['Venue', 'Category', 'Latitude', 'Longitude']
  
print('Shape:',nearby_venues.shape)
nearby_venues.head()

Shape: (5, 4)


,Venue,Category,Latitude,Longitude
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Upper Beaches,Neighborhood,43.680563,-79.292869


#### Build function to display dataframe of venues on a map

In [23]:
def draw_map_and_venues(df_venues, start_lat, start_long):
    
        map = folium.Map(location=[start_lat, start_long], zoom_start=15)

        # draw markers on map
        for lat, lng, categories, name in zip(df_venues['Latitude'], 
                                              df_venues['Longitude'], 
                                              df_venues['Category'], 
                                              df_venues['Venue']):
            
            label = '{}, {}'.format(categories, name)
            
            label = folium.Popup(label, parse_html=True)
            
            folium.CircleMarker(
                [lat, lng],
                radius=5,
                popup=label,
                color='green',
                fill=True,
                fill_color='#31cc77',
                fill_opacity=0.7,
                parse_html=False).add_to(map)  
            
        return map

#### Display map with venues within the Bourough

In [24]:
# evaluate a center of venues
latitude = nearby_venues['Latitude'].median()
longitude = nearby_venues['Longitude'].median()

#draw map with venues
draw_map_and_venues(nearby_venues, latitude, longitude)

## Explore all venues across all neighborhoods

#### Build function, which automates all the steps performed before along a list of coordinates

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Latitude', 
                  'Longitude', 
                  'Category']
    
    return(nearby_venues)

#### Get all venues from foursquare and process response

_**Note:** The FOURSCARE_FULL_ONLINE is used to reduce the number of API calls during development_

In [26]:
if FOURSCARE_FULL_ONLINE:
    # Pull data from foursquare
    selected_neighborhood_venues = getNearbyVenues(names=selected_neighborhood['Neighborhood'],
                                                    latitudes=selected_neighborhood['Latitude'],
                                                    longitudes=selected_neighborhood['Longitude']
                                                   )
    # Store result in file
    selected_neighborhood_venues.to_csv('capstone-data-package-part-3-neighborhood_venues.csv', index = False)
    print("Result pulled, processed and saved.")
else:
    # Load data from file instead from foursquare
    selected_neighborhood_venues = pd.read_csv('capstone-data-package-part-3-neighborhood_venues.csv')
    print("Result Loaded.")
    
print('Shape:',selected_neighborhood_venues.shape)
selected_neighborhood_venues.head()

Result Loaded.
Shape: (1685, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


#### Build map with all venues in the selected neighborhoods

In [27]:
# evaluate a center of venues
latitude = selected_neighborhood_venues['Latitude'].median()
longitude = selected_neighborhood_venues['Longitude'].median()

# draw map with venues
draw_map_and_venues(selected_neighborhood_venues, latitude, longitude)

#### Check how many venues were returned for per neighborhood

In [28]:
selected_neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Latitude,Longitude,Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17


#### Evaluate the number of venues categories listed

In [29]:
print('There are {} uniques categories.'.format(len(selected_neighborhood_venues['Category'].unique())))

There are 232 uniques categories.


## Analyze the Neighborhoods

In [30]:
# one hot encoding
venues_onehot = pd.get_dummies(selected_neighborhood_venues[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = selected_neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

print("Shape: ",venues_onehot.shape)
venues_onehot.head()

Shape:  (1685, 232)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
print("Shape: ", venues_grouped.shape)
venues_grouped.head()

Shape:  (38, 232)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.0,0.0,0.010000,0.0,0.03,0.0,0.0000,0.000000,0.0,0.000000,0.030000,0.0,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.01,0.000000,0.01,0.020000,0.0000,0.0,0.01,0.03,0.010000,0.0,0.000000,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.070000,0.0,0.0,0.0,0.01,0.000000,0.0000,0.020000,0.000000,0.02,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0

#### Create a function to sort the venues in descending order.

In [32]:
def return_most_common_venues(row, num_top_venues):
    
    #remove first row
    row_categories = row.iloc[1:]
    
    #sort rows
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    #sort return only the defined number of entries
    return row_categories_sorted.index.values[0:num_top_venues]

#### Put the top 5 common venuwa into *pandas* dataframe

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe with the new columns
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

# process all neighborhoods
for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

print('Shape:', neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

Shape: (38, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Asian Restaurant,Salad Place,Restaurant,Bar,Bakery,Burger Joint
1,Berczy Park,Coffee Shop,Beer Bar,Steakhouse,Bakery,Farmers Market,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Yoga Studio,Pet Store,Restaurant,Italian Restaurant,Intersection,Burrito Place,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Skate Park,Burrito Place,Recording Studio,Fast Food Restaurant,Auto Workshop,Farmers Market,Spa,Pizza Place,Restaurant,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Rental Car Location,Harbor / Marina,Boat or Ferry,Bar,Airport Gate,Airport Food Court


## Cluster Neighborhoods

#### Run k-means to cluster the neighborhood

In [34]:
# set number of clusters
kclusters = 5

venues_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Create a new dataframe which includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# remove clustering labels in case the column is already there
# neighborhoods_venues_sorted.drop('Cluster', axis=1, inplace=True)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

# Prepare dataframe to merge with cordinates
neighborhoods_merged = selected_neighborhood

# merge neighborhoods_venues_sorted with df_explore to add latitude/longitude for each neighborhood
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhoods_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,Other Great Outdoors,Pub,Trail,Health Food Store,Wings Joint,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bookstore,Bubble Tea Shop,Caribbean Restaurant,Brewery
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0.0,Sandwich Place,Park,Sushi Restaurant,Fast Food Restaurant,Steakhouse,Ice Cream Shop,Fish & Chips Shop,Burrito Place,Pizza Place,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Bookstore,Sandwich Place,Brewery,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2.0,Park,Gym / Fitness Center,Bus Line,Swim School,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [36]:
# drop rows undifined clusters
#neighborhoods_merged = neighborhoods_merged[~neighborhoods_merged['Cluster Label'].isnull()]

neighborhoods_merged = neighborhoods_merged.dropna()

# ensure cluster labels are int (can get changed due to NaN entries)
neighborhoods_merged['Cluster'] = neighborhoods_merged['Cluster'].astype(int)

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged['Latitude'],
                                  neighborhoods_merged['Longitude'], 
                                  neighborhoods_merged['Neighborhood'], 
                                  neighborhoods_merged['Cluster']):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyze Clusters

### Cluster 0: 

In [38]:
print('Shape:', neighborhoods_merged.shape)

Shape: (38, 16)


In [39]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster'] == 0, neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Other Great Outdoors,Pub,Trail,Health Food Store,Wings Joint,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
1,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bookstore,Bubble Tea Shop,Caribbean Restaurant,Brewery
2,"The Beaches West, India Bazaar",0,Sandwich Place,Park,Sushi Restaurant,Fast Food Restaurant,Steakhouse,Ice Cream Shop,Fish & Chips Shop,Burrito Place,Pizza Place,Food & Drink Shop
3,Studio District,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Bookstore,Sandwich Place,Brewery,Cheese Shop
5,Davisville North,0,Hotel,Pizza Place,Breakfast Spot,Sandwich Place,Food & Drink Shop,Park,Gym,Clothing Store,Comfort Food Restaurant,Deli / Bodega
6,North Toronto West,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Park,Chinese Restaurant,Cosmetics Shop,Mexican Restaurant,Burger Joint,Rental Car Location,Restaurant
7,Davisville,0,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Gym,Café,Sushi Restaurant,Coffee Shop,Fried Chicken Joint,Farmers Market
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Pub,Fried Chicken Joint,Light Rail Station,Restaurant,Liquor Store,Sports Bar,Bagel Shop,Supermarket,Pizza Place
11,"Cabbagetown, St. James Town",0,Coffee Shop,Restaurant,Pizza Place,Flower Shop,Italian Restaurant,Pub,Bakery,Café,Liquor Store,Deli / Bodega
12,Church and Wellesley,0,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Mediterranean Restaurant,Gastropub,Gym,Hotel,Café


### Cluster 1: 

In [40]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster'] == 1, neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,1,Park,Trail,Playground,Cuban Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
23,"Forest Hill North, Forest Hill West",1,Park,Jewelry Store,Trail,Sushi Restaurant,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


### Cluster 2: 

In [41]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster'] == 2, neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,2,Park,Gym / Fitness Center,Bus Line,Swim School,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster 3: 

In [42]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster'] == 3, neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,3,Health & Beauty Service,Garden,Wings Joint,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster 4: 

In [43]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster'] == 4, neighborhoods_merged.columns[[2] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",4,Restaurant,Playground,Intersection,Wings Joint,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


## Save result to CSV File

In [44]:
# Save to CSV File
neighborhoods_merged.to_csv("capstone-data-package-part-3-final.csv")